# Домашнее задание к вебинару 13. Transformers

Дообучение модели с помощью PyTorch

In [1]:
!pip install datasets
!pip install transformers
!pip install torchmetrics
!pip install torchvision

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 362 kB 8.8 MB/s 
     |████████████████████████████████| 212 kB 48.2 MB/s 
     |████████████████████████████████| 140 kB 59.8 MB/s 
     |████████████████████████████████| 1.1 MB 54.2 MB/s 
     |████████████████████████████████| 101 kB 12.2 MB/s 
     |████████████████████████████████| 596 kB 43.8 MB/s 
     |████████████████████████████████| 127 kB 54.6 MB/s 
     |████████████████████████████████| 94 kB 1.7 MB/s 
     |████████████████████████████████| 271 kB 56.4 MB/s 
     |████████████████████████████████| 144 kB 56.4 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
ERROR: pip's dep

In [2]:
import pandas as pd

### Загрузка данных

In [3]:
from datasets import load_dataset

corpus_train = load_dataset('merionum/ru_paraphraser', data_files='train.jsonl')
corpus_test = load_dataset('merionum/ru_paraphraser', data_files='test.jsonl')

Using custom data configuration merionum--ru_paraphraser-4a63244a00d42f51


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/merionum___json/merionum--ru_paraphraser-4a63244a00d42f51/0.0.0/da492aad5680612e4028e7f6ddc04b1dfcec4b64db470ed7cc5f2bb265b9b6b5. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Using custom data configuration merionum--ru_paraphraser-4ab1b249bc6d5bf7


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/merionum___json/merionum--ru_paraphraser-4ab1b249bc6d5bf7/0.0.0/da492aad5680612e4028e7f6ddc04b1dfcec4b64db470ed7cc5f2bb265b9b6b5. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [4]:
corpus_train, corpus_test

(DatasetDict({
    train: Dataset({
        features: ['id', 'id_1', 'id_2', 'text_1', 'text_2', 'class'],
        num_rows: 7227
    })
}),
 DatasetDict({
    train: Dataset({
        features: ['id', 'id_1', 'id_2', 'text_1', 'text_2', 'class'],
        num_rows: 1924
    })
}))

In [5]:
df_train = pd.DataFrame(corpus_train['train'])
df_train['text'] =  list(zip(df_train['text_1'], df_train['text_2']))
df_train = df_train[:2000]
df_train.head()

,id,id_1,id_2,text_1,text_2,class,text
0,1,201,8159,Полицейским разрешат стрелять на поражение по ...,Полиции могут разрешить стрелять по хулиганам ...,0,(Полицейским разрешат стрелять на поражение по...
1,2,202,8158,Право полицейских на проникновение в жилище ре...,Правила внесудебного проникновения полицейских...,0,(Право полицейских на проникновение в жилище р...
2,3,273,8167,Президент Египта ввел чрезвычайное положение в...,Власти Египта угрожают ввести в стране чрезвыч...,0,(Президент Египта ввел чрезвычайное положение ...
3,4,220,8160,Вернувшихся из Сирии россиян волнует вопрос тр...,Самолеты МЧС вывезут россиян из разрушенной Си...,-1,(Вернувшихся из Сирии россиян волнует вопрос т...
4,5,223,8160,В Москву из Сирии вернулись 2 самолета МЧС с р...,Самолеты МЧС вывезут россиян из разрушенной Си...,0,(В Москву из Сирии вернулись 2 самолета МЧС с ...


In [6]:
#Классы: 1 -- точные парафразы, 0 -- почти парафразы, -1 -- непарафразы
df_train['class'].value_counts()

0     745
1     642
-1    613
Name: class, dtype: int64

In [7]:
df_test = pd.DataFrame(corpus_test['train'])
df_test['text'] =  list(zip(df_test['text_1'], df_test['text_2']))
df_test = df_test[:500]
df_test.head()

,id,id_1,id_2,text_1,text_2,class,text
0,25349,34420,34421,Цены на нефть восстанавливаются,Парламент Словакии поблагодарил народы бывшего...,-1,"(Цены на нефть восстанавливаются, Парламент Сл..."
1,25360,34438,34439,"""Гоголь-центр"" покажет видеозапись скандальног...",Кехман запретил «Гоголь-центру» показывать вид...,-1,"(""Гоголь-центр"" покажет видеозапись скандально..."
2,25382,34475,34476,Агент: РФС вновь задерживает зарплату Фабио Ка...,СМИ: Агент Фабио Капелло грозится подать в суд...,-1,(Агент: РФС вновь задерживает зарплату Фабио К...
3,25423,34494,34519,День Победы в Москве обещает выдаться облачным,Любляна отпразднует День Победы вместе с Москвой,-1,(День Победы в Москве обещает выдаться облачны...
4,25430,34531,34532,Посол РФ в США: Россия будет бороться с попытк...,Правительство запланировало заработать на лоте...,-1,(Посол РФ в США: Россия будет бороться с попыт...


In [8]:
#LabelEncoder
y_train = df_train['class'].values
y_test = df_test['class'].values

from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(y_train)

label_list = le.classes_  #array(['-1', '0', '1'], dtype=object)
label_list

y_train = le.transform(y_train) #array([1, 1, 1, ..., 0, 0, 0])
y_test = le.transform(y_test)

### Подготовка данных для загрузки в модель

In [9]:
# Создаем класс датасета

import torch
class ParaphraseDataset(torch.utils.data.Dataset):
    
    def __init__(self, txts, classes):
        self._classes = classes
        self._txts = txts
    # функция для определения длины  (количества объектов)   
    def __len__(self):
        return len(self._txts)
    # фукнция для взятия элемента по индексу
    def __getitem__(self, index):
        return self._txts[index], self._classes[index]

In [10]:
#Создаем датасеты
# передаем туда столбец, где вместе оба предложения
train_dataset = ParaphraseDataset(df_train['text'], y_train)
test_dataset = ParaphraseDataset(df_test['text'], y_test)

# Создаем из датасетов даталоадеры (загрузчики данных)
train_loader = torch.utils.data.DataLoader(train_dataset,
                          batch_size=4,
                          shuffle=True,
                          num_workers=0)
test_loader = torch.utils.data.DataLoader(test_dataset,
                          batch_size=4,
                          shuffle=False,
                          num_workers=0)

In [11]:
# Протестируем работу даталоадера

for txt, cls in train_loader:
    print(txt[0])
    print(txt[1])
    print(len(txt[0]))
    print(cls)
    break

('Земфира представила новый альбом.', 'Сбербанк отказывается принимать чеки American Express.', 'Поиски детей в Туве приостановлены из-за опасности схода новой лавины.', 'Тело сгоревшего "охотника за полицейскими" так и не было найдено.')
('Новый альбом Земфиры появился в Сети.', 'Сбербанк откажется от чеков American Express.', 'В Туве приостановили поиски попавших под лавину подростков.', 'В сгоревшем убежище охотника за полицейскими найдены человеческие останки.')
4
tensor([1, 2, 1, 1])


### Загрузка и подготовка модели

In [12]:
from transformers import AutoModelForSequenceClassification, BertTokenizer

model_name = 'cointegrated/rubert-base-cased-dp-paraphrase-detection'

# Загружаем модель и токенайзер
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = BertTokenizer.from_pretrained(model_name)

Downloading:   0%|          | 0.00/1.09k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/679M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.57M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/406 [00:00<?, ?B/s]

In [13]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

В модели для нас не совпадает последний слой, т.к. у нас 3 класса (в модели 2).
Надо исправить последний слой (количество выходов)

In [14]:
#изменяем количество выходов в последнем слое классификации
import torch
import torch.nn as nn

model.classifier = nn.Linear(in_features=768, out_features=3, bias=True)

# проверим, все ли верно теперь
print(model)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

Отлично, переходим к обучению

In [15]:
import torchmetrics
test_f1 = torchmetrics.F1Score()

for test_input, test_class in test_loader:
    with torch.no_grad():
        batch = tokenizer(test_input[0], test_input[1], padding='max_length', max_length=64, truncation=True, return_tensors="pt")
        # print(batch)
        proba = torch.softmax(model(**batch).logits, -1) #torch.Size([B, 3])
        preds = torch.argmax(proba,dim=1) #torch.Size([B])
        target = test_class #torch.Size([B])
        
        test_f1(preds, target)

print(f'test f1_score: {test_f1.compute().item():.3f}')

test f1_score: 0.568


Пробуем дообучить модель

In [16]:
print("Parameters full train:", sum([param.nelement() for param in model.parameters()]))
print("Parameters for learning:", sum([param.nelement() for param in model.classifier.parameters()]))

Parameters full train: 177855747
Parameters for learning: 2307


In [17]:
#компиляция модели
criterion = nn.CrossEntropyLoss()
# обучаем последний слой классификации
optimizer = torch.optim.Adam(model.classifier.parameters(), lr=0.001)  

In [ ]:
#дообучение модели и подсчет метрик на тренировочном и валидационном датасетах
import torch
train_f1 = torchmetrics.F1Score()
test_f1 = torchmetrics.F1Score()

epochs = 8

for epoch in range(epochs):

    # обучение
    model.train()
    total_loss_train = 0.0
    #проходимся по каждому батчу даталоадера
    for train_input, train_class in train_loader:
        batch = tokenizer(train_input[0], train_input[1], padding='max_length', max_length=64, truncation=True, return_tensors="pt")
        proba = torch.softmax(model(**batch).logits, -1) #torch.Size([B, 3])
        target = train_class  # torch.Size([B])
        
        #loss
        batch_loss = criterion(proba, target)
        total_loss_train += batch_loss.item()

        #f1
        preds = torch.argmax(proba,dim=1) #torch.Size([B]))
        train_f1(preds, target)
        
        model.zero_grad()
        batch_loss.backward()
        optimizer.step()
    # валидация        
    model.eval()
    total_loss_test = 0.0
    for test_input, test_class in test_loader:
        batch = tokenizer(test_input[0], test_input[1], padding='max_length', max_length=64, truncation=True, return_tensors="pt")
        proba = torch.softmax(model(**batch).logits, -1) #torch.Size([B, 3])
        target = test_class  # torch.Size([B])
        
        #loss
        batch_loss = criterion(proba, target)
        total_loss_test += batch_loss.item()
       
        #f1
        preds = torch.argmax(proba,dim=1) #torch.Size([B]))
        test_f1(preds, target)
            
    print(
        f'Epochs: {epoch + 1} | Train Loss: {total_loss_train / len(train_dataset): .3f} \
        | Train f1: {train_f1.compute().item(): .3f} \
        | Test Loss: {total_loss_test / len(test_dataset): .3f} \
        | Test f1: {test_f1.compute().item(): .3f}')
    # сбрасываем расчет метрик, т.к. он имеется свойство накопления
    train_f1.reset()
    test_f1.reset()

Epochs: 1 | Train Loss:  0.211         | Train f1:  0.692         | Test Loss:  0.225         | Test f1:  0.624
Epochs: 2 | Train Loss:  0.211         | Train f1:  0.688         | Test Loss:  0.215         | Test f1:  0.678


In [ ]:
# рассчитаем метрику заново уже на дообученной модели
test_f1 = torchmetrics.F1Score()
model.eval()

for test_input, test_class in test_loader:
    batch = tokenizer(test_input[0], test_input[1], padding='max_length', max_length=64, truncation=True, return_tensors="pt")
    proba = torch.softmax(model(**batch).logits, -1) #torch.Size([B, 3])
    preds = torch.argmax(proba,dim=1) #torch.Size([B])
    target = test_class #torch.Size([B])
        
    test_f1(preds, target)

print(f'test f1_score: {test_f1.compute().item():.3f}')

**Вывод:** дообучение последнего классификационного слоя модели привело к улучшению метрики.